In [ ]:
### Intall all necessary dependencies ###
!pip install keras

In [ ]:
!pip install tensorflow

In [ ]:
!pip install tqdm

In [1]:
### get all necessary imports ###
import pandas as pd # provide sql-like data manipulation tools. very handy.
pd.options.mode.chained_assignment = None
import numpy as np # high dimensional vector computing library.
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Activation
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer
from sklearn.preprocessing import scale
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(return_indices=True)
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()

#create a list of english stopwords
#add words with no info used in twitter
stop_words = set(stopwords.words('english')) 
stop_words.add("http")
stop_words.add("https")
stop_words.add("co")
stop_words.add("amp")

C:\Users\nat\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\nat\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [168]:
### get the data from the excel file ###
def ingest():
    data = pd.read_excel('twitter_data_w_sent.xlsx', sheet_name='data')
    
    ### ### ### USE THIS FOR SENTIMENT CAN BE 1 POINT AWAY ### ### ###
    ### ### ### LARGER SAMPLE SIZE ### ### ###
    
    #data = data[data['standard-dev'] != 1]
    
    ### ### ### USE THIS FOR SENTIMENT MUST BE EQuAL ### ### ###
    ### ### ### SMALLER SAMPLE SIZE ### ### ###
    
    data = data[data['standard-dev'] == 0]
    
    ### #### ### ### ###
    data.drop(['time' , 'sent-1' , 'sent-2' , 'standard-dev'] , axis=1 , inplace=True)
    print('dataset loaded with shape', data.shape)  
    return data

def ingest_sent_one():
    data = pd.read_excel('twitter_data_w_sent.xlsx', sheet_name='data')
        
    data.drop(['time' , 'average' , 'sent-2' , 'standard-dev'] , axis=1 , inplace=True)
    print('dataset loaded with shape', data.shape)  
    data.columns = [ 'average' , 'text']
    return data

def ingest_sent_two():
    data = pd.read_excel('twitter_data_w_sent.xlsx', sheet_name='data')
    
    data.drop(['time' , 'sent-1' , 'average' , 'standard-dev'] , axis=1 , inplace=True)
    print('dataset loaded with shape', data.shape)  
    data.columns = [ 'average' , 'text']
    return data

data = ingest()
#data = ingest_sent_one() #### USE THIS TO TEST SENT ONE
#data = ingest_sent_two() ### USE THIS TO TEST SENT TWO

### function to tokenize the tweets and remove stop words ###
def tokenize(tweet):
    try:
        tokens = tokenizer.tokenize(tweet)
        new_tokens = list()
        for w in tokens:
            if (not w in stop_words):
                new_tokens.append(w)
        return new_tokens
    except:
        return 'NC'
    
### this functions actually runs the tokenize function above ###
def postprocess(data, n=1000):
    data = data.head(n)
    data['tokens'] = data['text'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

data = postprocess(data)

### separate the data to test and train ###
x_train, x_test, y_train, y_test = train_test_split(np.array(data.tokens),
                                                    np.array(data.average), test_size=0.2)

### give the tweets a label
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')
x_test = labelizeTweets(x_test, 'TEST')

### vectorize the words in tweets###
n_dim = 200
tweet_w2v = Word2Vec(size=n_dim, min_count=1)
tweet_w2v.build_vocab([x.words for x in tqdm(x_train)])
tweet_w2v.train([x.words for x in tqdm(x_train)] , total_words=tweet_w2v.corpus_count , epochs=9)

print('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in x_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print('vocab size :', len(tfidf))

def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweet_w2v[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=n_dim))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['accuracy'])

model.fit(train_vecs_w2v, y_train, epochs=8, batch_size=10, verbose=2)

score = model.evaluate(test_vecs_w2v, y_test, batch_size=10, verbose=2)
print('ACC: ' , score[1])

dataset loaded with shape (625, 2)


progress-bar: 100%|███████████████████████████████████████████████████████████████| 625/625 [00:00<00:00, 12288.25it/s]
0it [00:00, ?it/s]C:\Users\nat\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
500it [00:00, 100313.40it/s]
125it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<?, ?it/s]


building tf-idf matrix ...
vocab size : 89


0it [00:00, ?it/s]C:\Users\nat\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
500it [00:00, 6184.81it/s]
125it [00:00, 3686.20it/s]


Epoch 1/8
 - 1s - loss: 0.7255 - acc: 0.4600
Epoch 2/8
 - 0s - loss: 0.5499 - acc: 0.5000
Epoch 3/8
 - 0s - loss: 0.5119 - acc: 0.5060
Epoch 4/8
 - 0s - loss: 0.4973 - acc: 0.5100
Epoch 5/8
 - 0s - loss: 0.4893 - acc: 0.5120
Epoch 6/8
 - 0s - loss: 0.4833 - acc: 0.5180
Epoch 7/8
 - 0s - loss: 0.4786 - acc: 0.5240
Epoch 8/8
 - 0s - loss: 0.4751 - acc: 0.5280
ACC:  0.5840000009536743


In [169]:
### get the data from the excel file ###
def ingest():
    data = pd.read_excel('twitter_data_w_sent.xlsx', sheet_name='data')
    
    ### ### ### USE THIS FOR SENTIMENT CAN BE 1 POINT AWAY ### ### ###
    ### ### ### LARGER SAMPLE SIZE ### ### ###
    
    #data = data[data['standard-dev'] != 1]
    
    ### ### ### USE THIS FOR SENTIMENT MUST BE EQuAL ### ### ###
    ### ### ### SMALLER SAMPLE SIZE ### ### ###
    
    data = data[data['standard-dev'] == 0]
    
    ### #### ### ### ###
    data.drop(['time' , 'sent-1' , 'sent-2' , 'standard-dev'] , axis=1 , inplace=True)
    print('dataset loaded with shape', data.shape)  
    return data

def ingest_sent_one():
    data = pd.read_excel('twitter_data_w_sent.xlsx', sheet_name='data')
        
    data.drop(['time' , 'average' , 'sent-2' , 'standard-dev'] , axis=1 , inplace=True)
    print('dataset loaded with shape', data.shape)  
    data.columns = [ 'average' , 'text']
    return data

def ingest_sent_two():
    data = pd.read_excel('twitter_data_w_sent.xlsx', sheet_name='data')
    
    data.drop(['time' , 'sent-1' , 'average' , 'standard-dev'] , axis=1 , inplace=True)
    print('dataset loaded with shape', data.shape)  
    data.columns = [ 'average' , 'text']
    return data

data = ingest()
#data = ingest_sent_one() #### USE THIS TO TEST SENT ONE
#data = ingest_sent_two() ### USE THIS TO TEST SENT TWO

### function to tokenize the tweets and remove stop words ###
def tokenize(tweet):
    try:
        tokens = tokenizer.tokenize(tweet)
        new_tokens = list()
        for w in tokens:
            if (not w in stop_words):
                new_tokens.append(w)
        return new_tokens
    except:
        return 'NC'
    
### this functions actually runs the tokenize function above ###
def postprocess(data, n=1000):
    data = data.head(n)
    data['tokens'] = data['text'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

data = postprocess(data)

### separate the data to test and train ###
x_train, x_test, y_train, y_test = train_test_split(np.array(data.tokens),
                                                    np.array(data.average), test_size=0.2)

### give the tweets a label
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')
x_test = labelizeTweets(x_test, 'TEST')

### vectorize the words in tweets###
n_dim = 200
tweet_w2v = Word2Vec(size=n_dim, min_count=1)
tweet_w2v.build_vocab([x.words for x in tqdm(x_train)])
tweet_w2v.train([x.words for x in tqdm(x_train)] , total_words=tweet_w2v.corpus_count , epochs=9)

print('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in x_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print('vocab size :', len(tfidf))

def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweet_w2v[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=n_dim))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['accuracy'])

### OVERSAMPLE ###

X = train_vecs_w2v
y = y_train

X_ros, y_ros = ros.fit_sample(X, y)

### OVERSAMPLE ###

model.fit(X_ros, y_ros, epochs=8, batch_size=10, verbose=2)

score = model.evaluate(test_vecs_w2v, y_test, batch_size=10, verbose=2)
print('ACC: ' , score[1])

dataset loaded with shape (625, 2)


progress-bar: 100%|████████████████████████████████████████████████████████████████| 625/625 [00:00<00:00, 9641.01it/s]
0it [00:00, ?it/s]C:\Users\nat\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
500it [00:00, 99707.70it/s]
125it [00:00, 125517.84it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<?, ?it/s]


building tf-idf matrix ...
vocab size : 86


0it [00:00, ?it/s]C:\Users\nat\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
500it [00:00, 5628.85it/s]
125it [00:00, 4636.97it/s]


Epoch 1/8
 - 1s - loss: 0.7332 - acc: 0.3695
Epoch 2/8
 - 0s - loss: 0.6159 - acc: 0.4121
Epoch 3/8
 - 0s - loss: 0.5674 - acc: 0.4587
Epoch 4/8
 - 0s - loss: 0.5461 - acc: 0.4703
Epoch 5/8
 - 0s - loss: 0.5214 - acc: 0.5155
Epoch 6/8
 - 0s - loss: 0.5022 - acc: 0.5439
Epoch 7/8
 - 0s - loss: 0.4893 - acc: 0.5401
Epoch 8/8
 - 0s - loss: 0.4738 - acc: 0.5478
ACC:  0.5040000092983246


In [170]:
### get the data from the excel file ###
def ingest():
    data = pd.read_excel('twitter_data_w_sent.xlsx', sheet_name='data')
    
    ### ### ### USE THIS FOR SENTIMENT CAN BE 1 POINT AWAY ### ### ###
    ### ### ### LARGER SAMPLE SIZE ### ### ###
    
    #data = data[data['standard-dev'] != 1]
    
    ### ### ### USE THIS FOR SENTIMENT MUST BE EQuAL ### ### ###
    ### ### ### SMALLER SAMPLE SIZE ### ### ###
    
    data = data[data['standard-dev'] == 0]
    
    ### #### ### ### ###
    data.drop(['time' , 'sent-1' , 'sent-2' , 'standard-dev'] , axis=1 , inplace=True)
    print('dataset loaded with shape', data.shape)  
    return data

def ingest_sent_one():
    data = pd.read_excel('twitter_data_w_sent.xlsx', sheet_name='data')
        
    data.drop(['time' , 'average' , 'sent-2' , 'standard-dev'] , axis=1 , inplace=True)
    print('dataset loaded with shape', data.shape)  
    data.columns = [ 'average' , 'text']
    return data

def ingest_sent_two():
    data = pd.read_excel('twitter_data_w_sent.xlsx', sheet_name='data')
    
    data.drop(['time' , 'sent-1' , 'average' , 'standard-dev'] , axis=1 , inplace=True)
    print('dataset loaded with shape', data.shape)  
    data.columns = [ 'average' , 'text']
    return data

#data = ingest()
#data = ingest_sent_one() #### USE THIS TO TEST SENT ONE
data = ingest_sent_two() ### USE THIS TO TEST SENT TWO

### function to tokenize the tweets and remove stop words ###
def tokenize(tweet):
    try:
        tokens = tokenizer.tokenize(tweet)
        new_tokens = list()
        for w in tokens:
            if (not w in stop_words):
                new_tokens.append(w)
        return new_tokens
    except:
        return 'NC'
    
### this functions actually runs the tokenize function above ###
def postprocess(data, n=1000):
    data = data.head(n)
    data['tokens'] = data['text'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

data = postprocess(data)

### separate the data to test and train ###
x_train, x_test, y_train, y_test = train_test_split(np.array(data.tokens),
                                                    np.array(data.average), test_size=0.2)

### give the tweets a label
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')
x_test = labelizeTweets(x_test, 'TEST')

### vectorize the words in tweets###
n_dim = 200
tweet_w2v = Word2Vec(size=n_dim, min_count=1)
tweet_w2v.build_vocab([x.words for x in tqdm(x_train)])
tweet_w2v.train([x.words for x in tqdm(x_train)] , total_words=tweet_w2v.corpus_count , epochs=9)

print('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in x_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print('vocab size :', len(tfidf))

def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweet_w2v[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=n_dim))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['accuracy'])

### OVERSAMPLE ###

X = train_vecs_w2v
y = y_train

X_rus, y_rus, id_rus = rus.fit_sample(X, y)

### OVERSAMPLE ###

model.fit(X_rus, y_rus, epochs=8, batch_size=10, verbose=2)

score = model.evaluate(test_vecs_w2v, y_test, batch_size=10, verbose=2)
print('ACC: ' , score[1])

dataset loaded with shape (959, 2)


progress-bar: 100%|███████████████████████████████████████████████████████████████| 959/959 [00:00<00:00, 10869.89it/s]
0it [00:00, ?it/s]C:\Users\nat\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
767it [00:00, 109852.52it/s]
192it [00:00, 96098.61it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████| 767/767 [00:00<?, ?it/s]


building tf-idf matrix ...
vocab size : 147


0it [00:00, ?it/s]C:\Users\nat\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
767it [00:00, 4836.63it/s]
192it [00:00, 3632.20it/s]


Epoch 1/8
 - 1s - loss: 0.7851 - acc: 0.3413
Epoch 2/8
 - 0s - loss: 0.6688 - acc: 0.3611
Epoch 3/8
 - 0s - loss: 0.6313 - acc: 0.3611
Epoch 4/8
 - 0s - loss: 0.6121 - acc: 0.3889
Epoch 5/8
 - 0s - loss: 0.5883 - acc: 0.3968
Epoch 6/8
 - 0s - loss: 0.5768 - acc: 0.4087
Epoch 7/8
 - 0s - loss: 0.5632 - acc: 0.4286
Epoch 8/8
 - 0s - loss: 0.5402 - acc: 0.4365
ACC:  0.5572916710128387
